### Mapbox

In [1]:
# 计算从公园出发n分钟内的amentity数目

In [2]:
# 1.加载城市边界数据

In [3]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx

# 选择适合的投影坐标系统
projected_crs = '3857'  # Web Mercator 投影

# 读取seattle 边界
# 加载seattle boundary
seattle_boundary = gpd.read_file("Seattle_City_Boundary.geojson")
seattle_boundary = seattle_boundary.to_crs(epsg=projected_crs)  


In [4]:
# 加载公园数据
parks_detail = gpd.read_file('Park_Details.geojson')
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,PMA,OWNER,LEASE,MAINT,...,ACQ_DATE,NAMEFLAG,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry
0,659873,EAST DUWAMISH GREENBELT,East Duwamish GS: S Spokane St,,3959401915,1179.0,3925,DPR,N,DPR,...,"Mon, 05 Feb 1990 00:00:00 GMT",9.0,"Thu, 08 Apr 2004 00:00:00 GMT",PROPERTY-EDUWS,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",9a608d6a-0194-46b0-ae98-8f29e5e15265,"POLYGON ((1273174.75 208703.265, 1273247.5 208..."
1,660537,EAST DUWAMISH GREENBELT,East Duwamish GS: S Spokane St,,5679501250,9957.0,3925,DPR,N,DPR,...,"Thu, 08 Jun 1995 00:00:00 GMT",9.0,"Tue, 28 Apr 1998 00:00:00 GMT",PROPERTY-EDUWS,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",635bcf21-7ca7-4d75-8f0b-cf8d84dda30d,"POLYGON ((1273608.076 210324.758, 1273776.532 ..."
2,660205,LAKE WASHINGTON BOULEVARD,Lake Washington Boulevard,,5249809999,16114.0,412,DPR,N,DPR,...,"Sat, 04 Feb 1911 00:00:00 GMT",9.0,"Mon, 20 May 2002 00:00:00 GMT",PROPERTY-LKWBLN,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",85f40e2f-9706-4001-ac59-0889f48da3ab,"POLYGON ((1284668.487 211643.908, 1284802.67 2..."
3,658877,HAMILTON VIEWPOINT PARK,Hamilton Viewpoint Park,1120 California Ave SW,9272202790,1945.0,454,DPR,N,DPR,...,"Thu, 01 Jan 2099 00:00:00 GMT",9.0,"Wed, 31 Mar 2004 00:00:00 GMT",PROPERTY-HAMVP,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",1b4161f5-dfcd-4f7b-bf1a-f18158df61d3,"POLYGON ((1257308.533 220799.055, 1257309.555 ..."
4,659541,DUWAMISH HEAD GREENBELT,Duwamish Head Greenbelt,,9151600225,1884.0,451,DPR,N,DPR,...,"Wed, 05 Sep 1990 00:00:00 GMT",9.0,"Wed, 31 Mar 2004 00:00:00 GMT",PROPERTY-DUWFM,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",205f3b84-1d36-46a4-b555-3cf6a4a62868,"POLYGON ((1258460.076 217645.437, 1258458.756 ..."


In [5]:
parks_detail = parks_detail.rename(columns = {'PMA':'TOTAL_AREA','USE_':'TYPE'})
parks_detail = parks_detail.to_crs(epsg=projected_crs)
parks_detail = parks_detail[parks_detail.is_valid]
parks_detail = gpd.clip(parks_detail,seattle_boundary)
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,ACQ_DATE,NAMEFLAG,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry
50,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Mon, 04 Jan 2010 00:00:00 GMT",9.0,"Wed, 06 Jan 2010 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737..."
1625,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Tue, 26 Mar 1996 00:00:00 GMT",9.0,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830..."
2591,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Tue, 26 Mar 1996 00:00:00 GMT",9.0,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739..."
992,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Tue, 26 Mar 1996 00:00:00 GMT",9.0,"Wed, 07 Apr 2004 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846..."
1523,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Fri, 11 Jul 1997 00:00:00 GMT",9.0,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8..."


In [6]:
parks_detail['centroid'] = parks_detail.geometry.centroid

In [7]:
# 2. 加载amentity数据

In [8]:
def ReadAmentityData(path,clip_mask,projected_crs):
    # 读取
    amentity_gdf = gpd.read_file(path)
    # 转换相同crs
    if amentity_gdf.crs != projected_crs:
        amentity_gdf = amentity_gdf.to_crs(epsg=projected_crs) 
    # 裁剪数据
    amentity_gdf_clip = gpd.clip(amentity_gdf,clip_mask)
    return amentity_gdf_clip

In [9]:
# 2.1 读取学校数据
schools_gdf = ReadAmentityData(path = 'Seattle_Public_Schools_Sites.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
schools_gdf = schools_gdf[~schools_gdf['Status'].str.contains('Closed')]
schools_gdf.head()

,OBJECTID_1,OBJECTID,schID,schName,mapLabel,Status,esmshs,geometry
2,3,3,264,Rainier View,Rainier View,ELEM,ES,POINT (-13610274.057 6023884.827)
4,5,5,221,Emerson,Emerson,ELEM,ES,POINT (-13609769.121 6026514.487)
6,7,7,291,South Shore K-8,South Shore K-8,Option ELEM,ES,POINT (-13611257.794 6027985.217)
8,9,9,21,Rainier Beach,Rainier Beach,HS,HS,POINT (-13610573.163 6028039.258)
10,11,11,219,Dunlap,Dunlap,ELEM,ES,POINT (-13611552.404 6028190.661)


In [10]:
# 2.2 读取landmark数据
landmarks_gdf = ReadAmentityData(path = 'Landmarks_and_Government_Buildings.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
landmarks_gdf.head()

,OBJECTID,PERMANENT_IDENTIFIER,SOURCE_FEATUREID,SOURCE_DATASETID,SOURCE_DATADESC,SOURCE_ORIGINATOR,DATA_SECURITY,DISTRIBUTION_POLICY,LOADDATE,FTYPE,...,ADDRESSBUILDINGNAME,ADDRESS,CITY,STATE,ZIPCODE,GNIS_ID,FOOT_ID,COMPLEX_ID,GLOBALID,geometry
184874,1003832,{A4D79E6D-E90F-4E40-B933-5434EEA8FB65},None,2f97153f-59ba-442e-8b8e-c222ac07af34,780 TNMC Update 2/21/2023,U.S. Geological Survey,5.0,E4,"Tue, 21 Feb 2023 15:50:41 GMT",780,...,None,2721 Southwest Trenton Street,Seattle,WA,98126,2462269,None,None,5a46ac9d-5a03-46ba-a9af-a905d86c59f2,POINT (-13621845.468 6028039.808)
17019,1051652,{6E7174F1-8105-4D8E-8AA9-F6E12E7919CA},None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,5.0,E4,"Wed, 08 Mar 2023 21:53:12 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,1504959,None,None,566da1b2-9ab4-45ba-a686-4b0e4277a25a,POINT (-13621952.689 6030937.665)
171565,1471050,317a4f56-3ecd-4ca4-b73a-2eb43826af1f,None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,0.0,E4,"Wed, 08 Mar 2023 21:53:07 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,None,None,None,7c50bbdd-a30a-44df-a8c6-5b0541b235bc,POINT (-13622254.81 6031017.647)
171593,1471085,9957cb1b-a289-488f-ae3e-e9f2ffd50a9e,None,52740733-6aaf-4a55-967c-eb4358ba9a6d,820 TNMC Update 1/11/2024,U.S. Geological Survey,0.0,E4,"Thu, 11 Jan 2024 03:02:47 GMT",820,...,None,22nd Avenue South,Seattle,WA,98108,None,None,None,996c8467-6c8b-4d03-a2d3-59af5d9364b6,POINT (-13614950.916 6031818.468)
184858,1003815,{A71622B7-BAA4-48B1-8CC7-D8A1DD759EE5},None,86aaf0c3-7986-4d31-a3a5-e7154a52221d,780 TNMC Update 3/11/2023,U.S. Geological Survey,5.0,E4,"Sat, 11 Mar 2023 02:46:35 GMT",780,...,None,620 South Orcas Street,Seattle,WA,98108,2462221,None,None,1a953dfe-4bec-47fd-8360-eed9eeb67d63,POINT (-13617253.003 6032669.504)


In [11]:
# 2.3 读取metro stop数据
metro_stop_gdf = ReadAmentityData(path = 'King_County_Metro_Transit_Stops.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
metro_stop_gdf.head()

,OBJECTID_1,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
13132,13133,13133,155,4,YES,None,55380,127316,ACT,REG,...,Beacon Ave S,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-asphlt,Single,1,4715,POINT (-13609919.788 6023459.351)
19051,19052,19052,155,5,YES,None,55380,127316,ACT,REG,...,Beacon Ave S,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-asphlt,Single,1,4715,POINT (-13609919.788 6023459.351)
13123,13124,13124,155,4,YES,None,55370,48188,ACT,REG,...,59th Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4707,POINT (-13609888.325 6023633.418)
20216,20217,20217,155,5,YES,None,55370,48188,ACT,REG,...,59th Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4707,POINT (-13609888.325 6023633.418)
843,844,844,155,4,YES,None,55430,51575,ACT,REG,...,51st Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4720,POINT (-13611040.48 6024869.901)


In [12]:
# 2.4 读取housing的数据
housing_gdf = ReadAmentityData(path = 'King_County_Assessor_Residential_Unit_Types_and_Sizes.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
housing_gdf = housing_gdf[housing_gdf['UNIT_TYPE'].str.contains('RES-2 Unit|RES-3 Unit|RES-4 Unit|Apartments', na=False)]
housing_gdf.rename(columns={'SQFTTOTLIVING:': 'square of living space'}, inplace=True)
housing_gdf.head()

,OBJECTID,ADDRESS,MAJOR,PIN,BLDGNBR,PRES_USE,SECTION_USE,UNIT_TYPE,NBRTHISTYPE,BEDROOMS,...,VILLNUMB,UV_NAME,UC_NAME,CRA_NAME,TRBL20,POINT_X,POINT_Y,LAT,LON,geometry
159065,159066,5542 S 119TH ST,334840,3348401367,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3014,1.287169e+06,184485.440210,47.496766,-122.262382,POINT (-13610184.472 6023538.253)
159060,159061,5548 S 119TH ST,334840,3348401365,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3014,1.287236e+06,184485.083255,47.496768,-122.262113,POINT (-13610154.515 6023538.649)
159068,159069,11650 54TH AVE S,334840,3348401416,1,Duplex,Residence (348),RES-2 Unit,2,3,...,0.0,Outside Villages,None,Rainier Beach,11901.3008,1.286355e+06,184860.328896,47.497752,-122.265702,POINT (-13610554.107 6023700.727)
159064,159065,5160 S AUGUSTA ST,334840,3348401584,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3004,1.285813e+06,185644.090451,47.499872,-122.267955,POINT (-13610804.885 6024050.138)
159199,159200,5154 S AUGUSTA ST,334840,3348401585,1,Duplex,Residence (348),RES-2 Unit,2,3,...,0.0,Outside Villages,None,Rainier Beach,11901.3004,1.285750e+06,185670.215399,47.499941,-122.268211,POINT (-13610833.382 6024061.408)


In [13]:
# 2.5 读取工作机会的数据
jobs_gdf = ReadAmentityData(path = 'Business_Points_of_Interest_Seattle1.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
jobs_gdf.rename(columns={'EMPNUM': 'Employers Number'}, inplace=True)
jobs_gdf.head()

,OBJECTID,CONAME,ADDR,CITY,STATE_NAME,STATE,ZIP,ZIP4,NAICS,NAICS_ALL,...,SOURCE,ESRI_PID,DESC_,LATITUDE,LONGITUDE,CreationDate,Creator,EditDate,Editor,geometry
1187,1188,Ideoclick Inc,1st Ave S,Seattle,Washington,WA,98104,4420,51821013,51821013,...,Data Axle,853edb450c88380e121dde3faa397513,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
1329,1330,Code Systems Corp,1st Ave S,Seattle,Washington,WA,98104,4420,51321005,51321005,...,Data Axle,c0c4e438b4811963fdd9d9762d060b7f,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
926,927,Graham Construction & Engineering,1st Ave S,Seattle,Washington,WA,98104,4420,23611512,"23611512, 23611505, 23799012",...,Data Axle,c7559040fecc57731ecb483b0126b15f,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
3941,3942,Sice Agencia Chile Sa,Occidental Ave S,Seattle,Washington,WA,98104,2869,99999004,99999004,...,Data Axle,3fe875f7c4ccdab459c1890f23cc8dcc,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
1911,1912,Five Six Eight First Ave,1st Ave S,Seattle,Washington,WA,98104,4419,99999004,99999004,...,Data Axle,814e6019f4011b4684243e8021abe270,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)


In [14]:
# 2.6 读取MHA数据（affordable housing）
MHA_gdf= gpd.read_file('Seattle_Mandatory_Housing_Affordability_MHA_Zones.geojson')
# 转换坐标系统
if MHA_gdf.crs != projected_crs:
   MHA_gdf = MHA_gdf.to_crs(epsg=projected_crs)  
MHA_gdf =  MHA_gdf[ MHA_gdf.geometry.is_valid]
# 裁剪
MHA_gdf_clip = gpd.clip(MHA_gdf,seattle_boundary )
MHA_gdf_clip['centroid'] = MHA_gdf_clip.geometry.centroid
MHA_gdf_clip.head()

,OBJECTID,ZONEID,ZONING,CONTRACT,ORDINANCE,EFFECTIVE,HISTORIC,PEDESTRIAN,SHORELINE,OVERLAY,...,PEDESTRIAN_PREV,SHORELINE_PREV,OVERLAY_PREV,LIGHTRAIL_PREV,BZONEID,PUBLIC_DESCRIPTION,CHAPTER,CHAPTER_LINK,geometry,centroid
671,672,5224,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13610031.832 6023559.014, -13610013...",POINT (-13609944.022 6023467.407)
691,692,5252,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608806.519 6024176.479, -13608824...",POINT (-13608748.347 6024221.13)
1677,1678,4957,LR1 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,2966.0,is a multifamily residential zone where reside...,Chapter 23.45,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13611047.866 6025230.025, -13610974...",POINT (-13611012.153 6025175.461)
1379,1380,4708,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608555.044 6025896.358, -13608555...",POINT (-13608593.266 6025912.224)
2030,2031,4714,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13609328.712 6026711.971, -13609324...",POINT (-13609354.246 6026733.578)


In [15]:
# 3. 计算xx分钟内可达的xx数目

In [16]:
# 3.1 使用等时线矩阵

In [22]:
# 3.1.1 获取等时线矩阵
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import numpy as np
import time
import requests

#这里用mapbox来计算
def get_isochrone_mapbox(centroid, profile='walking', minutes=5,access_token=""):
    lon, lat = centroid.x, centroid.y
    headers = {  
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36'  
}  
    url = f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/{lon},{lat}?contours_minutes={minutes}&access_token={access_token}"
    retries = 3
    while retries > 0:
        response = requests.get(url)
        if response.status_code == 200:#正常请求
            data = response.json()
            if 'features' in data and len(data['features']) > 0:
                coordinates = data['features'][0]['geometry']['coordinates']
                polygon = Polygon(coordinates)
                return polygon
            else:
                print(f"No features found for location: ({lon}, {lat})")
                return None
        elif response.status_code == 429:  # 请求太多
            print("Rate limit exceeded. Retrying...")
            time.sleep(60)  # 等待一分钟后重试
            retries -= 1
        else:
            print(f"Failed to fetch isochrone data for location: ({lon}, {lat})")
            return None
        time.sleep(12)  # 每次请求后等待12秒（相当于每分钟5次请求）
    return None


In [23]:
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point

# 转换函数
def transform_point(_transformer,point):
    lon, lat = _transformer.transform(point.x, point.y)
    return Point(lon, lat)
# 获取中心点
parks_centroids = parks_detail.geometry.centroid


In [24]:
profile='walking'
travel_time = 10
# 获取等时线
accesstoken = 'pk.eyJ1Ijoiemh1b2FueXVhbnl1IiwiYSI6ImNtZ2htNXFmcTBoNGEycm9lbTY1MDZpYzMifQ.9XG1lOyIj4q5W5xNA3fw8g'
 # 获取等时线
isochrones =[]
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
# 这行代码使用 pyproj 库中的 Transformer.from_crs 方法创建了一个坐标转换对象 transformer。
# 该对象用于将 EPSG:3857 坐标系下的坐标转换为 EPSG:4326 坐标系下的坐标。
# always_xy=True：该参数确保输入和输出的坐标顺序始终为 x（经度）、y（纬度），避免因不同库或系统对坐标顺序的定义不同而产生混淆。


In [28]:
# 对于每一行获取等时矩阵
for index,row in parks_detail.iterrows():
    centroid = transform_point(transformer, row.geometry.centroid) 
    # 将中心点按照上面的函数从投影变成地理坐标
    # `row.geometry.centroid` 获取当前公园几何图形的中心点
    # `transform_point` 函数使用 `transformer` 对中心点的坐标进行转换
    isochrone = get_isochrone_mapbox(centroid, profile = profile,minutes = travel_time, access_token=accesstoken)
    isochrones.append(isochrone)

In [30]:
isochrones_gdf  = gpd.GeoDataFrame({'geometry': isochrones})
isochrones_gdf.head()

,geometry
0,"POLYGON ((-122.3712 47.50283, -122.3722 47.503..."
1,"POLYGON ((-122.37183 47.50265, -122.37234 47.5..."
2,"POLYGON ((-122.37058 47.50313, -122.37158 47.5..."
3,"POLYGON ((-122.37216 47.50227, -122.37316 47.5..."
4,"POLYGON ((-122.37283 47.50192, -122.37425 47.5..."


In [31]:
# 在进行空间分析之前，确保所有涉及的 GeoDataFrame 使用相同的 CRS，一定要用wgs84。
# 转换 CRS
isochrones_gdf.set_crs(epsg=4326, inplace=True,allow_override=True)
isochrones_gdf_proj = isochrones_gdf.to_crs(epsg=projected_crs)
parks_detail['isochrones'] = isochrones_gdf_proj['geometry']
parks_detail.set_geometry('geometry', inplace=True)
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones
50,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Wed, 06 Jan 2010 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893 6023634.084),"POLYGON ((-13623319.47 6031677.956, -13623585...."
1625,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.094 6023678.226),"POLYGON ((-13623401.623 6038683.571, -13623512..."
2591,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.82 6023732.238),"POLYGON ((-13616262.816 6035189.128, -13616374..."
992,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Wed, 07 Apr 2004 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.34 6023754.705),"POLYGON ((-13611805.027 6028358.78, -13611916...."
1523,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.852 6023768.189),"POLYGON ((-13619588.486 6033210.681, -13619699..."


In [32]:
# 存取中间项
parks_detail.to_csv(f'park_details_isochrones_{travel_time}_{profile}_gdf.csv', index=True)

In [33]:
# WKT（Well-Known Text）是一种用于表示几何对象的文本格式。
# 它是一个基于文本的格式，用于描述几何形状，例如点、线、多边形等。这种格式通常用于地理信息系统（GIS）中，作为一种标准的几何数据交换格式。
parks_detail['isochrones'] = parks_detail['isochrones'].to_wkt()
parks_detail['centroid'] = parks_detail['centroid'].to_wkt()
parks_detail.to_file(f'park_details_isochrones_{travel_time}_{profile}_gdf.geojson', driver='GeoJSON')

In [34]:
# 3.1.2 计算xx时间范围内xx的数目
def CalculateAmenityCountInIsochrones(amenity_gdf_clip,amenity_name,intersect_grid):
    # 执行空间连接,找到每个等时线范围内的设施
    amenity_within_isochrones = gpd.sjoin(amenity_gdf_clip,intersect_grid,how = 'inner',predicate = 'within')
    
    # 统计每个等时线范围内的设施数目
    amenity_counts = amenity_within_isochrones.groupby('index_right').size()
    
    # 创建一个新的列,填充统计结果
    amenity_index = f'{amenity_name}_count'
    intersect_grid[amenity_index] = intersect_grid.index.map(amenity_counts).fillna(0).astype(int)

    return intersect_grid

In [35]:
# 读取公园等时线文件
park_detail = gpd.read_file('park_details_isochrones_10_walking_gdf.geojson')
park_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,centroid,isochrones,geometry
0,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Wed, 06 Jan 2010 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...","POLYGON ((-13622737.379 6023640.776, -13622737..."
1,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...","POLYGON ((-13622809.153 6023663.187, -13622830..."
2,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...","POLYGON ((-13622749.842 6023792.899, -13622739..."
3,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Wed, 07 Apr 2004 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...","POLYGON ((-13622845.404 6023793.752, -13622846..."
4,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...","POLYGON ((-13622922.56 6023813.85, -13622922.8..."


In [36]:
parks_detail['isochrones'].dtype

dtype('O')

**关于from shapely import wkt**

parks_detail['isochrones'] = parks_detail['isochrones'].to_wkt()：这行代码将 parks_detail 数据框中 isochrones 列的几何对象转换为 Well-Known Text（WKT）格式的字符串。WKT 是一种用于表示几何对象的文本格式，方便存储和交换几何信息。[存储地理信息时会用到]

parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)：这行代码将 parks_detail 数据框中 isochrones 列的 WKT 字符串转换回几何对象。wkt.loads 是一个函数，用于解析 WKT 字符串并将其转换为相应的几何对象。[从文件读取地理信息时会用到]


In [37]:
# 读取等时矩阵
from shapely import wkt
# 第一次读取的时候
parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)
parks_detail['centroid'] = parks_detail['centroid'].apply(wkt.loads)


In [38]:
# create geodataframe
selected_columns = parks_detail[['OBJECTID','isochrones','centroid']]
parks_isochrones = gpd.GeoDataFrame(selected_columns,geometry = 'isochrones')
# geometry = 'isochrones'：这指定了 GeoDataFrame 中的哪一列将被用作几何列。几何列包含地理空间数据，如点、线、多边形等。
# 这里指定 'isochrones' 列作为几何列，意味着 selected_columns 中应该存在名为 'isochrones' 的列，
# 且该列的数据类型应该是 geopandas 支持的几何类型，如 shapely 中的 Point、LineString 或 Polygon 等。

parks_isochrones.set_crs(epsg=projected_crs,inplace=True)
parks_isochrones = parks_isochrones.to_crs(epsg=projected_crs)
parks_isochrones.head()

,OBJECTID,isochrones,centroid
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662)
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553)
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333)
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958)
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662)


In [39]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = schools_gdf,amenity_name = 'Schools',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),1
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),0


In [40]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = landmarks_gdf,amenity_name = 'Landmarks',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1,0
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),1,0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),0,0


In [41]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = metro_stop_gdf,amenity_name = 'Metros',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1,0,58
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),1,0,26
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),0,0,0


In [42]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = housing_gdf,amenity_name = 'Housings',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,Housings_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1,0,58,21
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),1,0,26,6
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),0,0,0,0


In [43]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = MHA_gdf_clip,amenity_name = 'Affordable_Housings',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6
1625,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1,0,58,21,21
2591,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),1,0,26,6,0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34
1523,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),0,0,0,0,0


In [44]:
# 因为每个工作点,对应了多个工作机会
jobs_with_isochrones = gpd.sjoin(jobs_gdf,parks_isochrones,how='inner',predicate = 'within')
jobs_with_isochrones.head()

,OBJECTID_left,CONAME,ADDR,CITY,STATE_NAME,STATE,ZIP,ZIP4,NAICS,NAICS_ALL,...,Editor,geometry,index_right,OBJECTID_right,centroid,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count
617,618,Bodypoint Designs Inc,1st Ave S,Seattle,Washington,WA,98104,3806,42345028,"42345028, 33911203, 33911320",...,ULI_Hines_Admin_Support,POINT (-13618124.356 6039988.904),658,660955,POINT (-13616105.686844 6032647.566674),0,6,428,1,10
617,618,Bodypoint Designs Inc,1st Ave S,Seattle,Washington,WA,98104,3806,42345028,"42345028, 33911203, 33911320",...,ULI_Hines_Admin_Support,POINT (-13618124.356 6039988.904),1995,659485,POINT (-13621806.173474 6059404.273841),0,6,428,1,10
2083,2084,IUNU,1st Ave S,Seattle,Washington,WA,98104,3832,54151103,54151103,...,ULI_Hines_Admin_Support,POINT (-13618124.356 6039988.904),658,660955,POINT (-13616105.686844 6032647.566674),0,6,428,1,10
2083,2084,IUNU,1st Ave S,Seattle,Washington,WA,98104,3832,54151103,54151103,...,ULI_Hines_Admin_Support,POINT (-13618124.356 6039988.904),1995,659485,POINT (-13621806.173474 6059404.273841),0,6,428,1,10
4851,4852,USC Events,1st Ave S,Seattle,Washington,WA,98104,2866,71131004,71131004,...,ULI_Hines_Admin_Support,POINT (-13618124.356 6039988.904),658,660955,POINT (-13616105.686844 6032647.566674),0,6,428,1,10


In [45]:
# 计算每个区域内employers数量的总和
aggregated_employers = jobs_with_isochrones.groupby('index_right').agg({'Employers Number':'sum'}).reset_index()

In [46]:
aggregated_employers.head()

,index_right,Employers Number
0,641,22122.0
1,642,14341.0
2,643,14835.0
3,644,12461.0
4,645,15362.0


In [47]:
# 将总和信息添加到parks_detail
parks_isochrones = parks_detail.merge(aggregated_employers,how = 'left',left_index = True,right_on = 'index_right')
# parks_detail：这是左表，即操作完成后保留其所有行的数据表。
# aggregated_employers：这是右表，用于提供要合并的数据。
# how = 'left'：指定连接方式为左连接，意味着左表 parks_detail 的所有行都会被保留，而右表中匹配的行将被合并进来；若右表中没有匹配的行，则合并后的对应列会填充 NaN 值。
# left_index = True：表示使用左表 parks_detail 的索引作为连接键。
# right_on = 'index_right'：表示使用右表 aggregated_employers 中的 index_right 列作为连接键。
parks_isochrones.head(5)

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,index_right,Employers Number
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",50,NaN
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",1625,NaN
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",2591,NaN
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",992,NaN
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",1523,NaN


In [48]:
# 重命名新列
parks_isochrones = parks_isochrones.rename(columns={'Employers Number': 'job_count'})

# 填充缺失值，并确保列的类型为整数
parks_isochrones['job_count'] = parks_isochrones['job_count'].fillna(0).astype(int)

# 删掉index_right这一列
parks_isochrones = parks_isochrones.drop(columns=['index_right'])

parks_isochrones.head()
parks_isochrones.to_csv(f'park_details_{profile}_{travel_time}_Amentities.csv')

In [50]:
# 计算从公园出发到最近的xx的距离

In [60]:
# 1. 筛选我们有的数据点（只选在10分钟范围内的）
# 2. 计算距离

# 计算每个公园到最近amenity的距离(米)
def calculate_nearest_amenity_distance(parks,amenity_within_isochrones):
    distance = []
    for _, park in parks.iterrows():
    #_, park：这是元组解包的语法。_ 是一个惯用的占位符，表示这个变量不会被使用，在这个上下文中，就是忽略行的索引。
    # park 是一个变量，它会依次被赋值为数据框中每一行的数据（即一个 Series 对象）。
    # 通过 park 可以访问该行的各个列的值，例如 park['latitude'] 可以获取该行的纬度值。
        park_point = park.geometry.centroid
        
        # 计算公园到筛选后amenity的距离
        distance_to_amenity = amenity_within_isochrones.geometry.distance(park_point)
        # distance(park_point)：这是 shapely 几何对象的一个方法，用于计算当前几何对象与另一个几何对象（这里是 park_point）之间的距离。
        # park_point 通常是一个 shapely 的 Point 对象，表示一个具体的地理点。
        
        # 找到最小距离
        nearest_distance = distance_to_amenity.min() if not distance_to_amenity.empty else float('inf')
        # float('inf')：如果 distances_to_amentity 为空，这部分代码会将正无穷大赋值给 nearest_distance。在实际应用中，正无穷大通常表示没有有效的距离数据。
        
        distance.append(nearest_distance)
    return distance

In [61]:
amenity_within_isochrones = gpd.sjoin(metro_stop_gdf,parks_isochrones,how = 'inner',predicate = 'within')
parks_isochrones['nearest_metro_distance'] = calculate_nearest_amenity_distance(parks = parks_detail,amenity_within_isochrones = amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,job_count,nearest_metro_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",0,10173.356557
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",0,10141.741316
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",0,10083.838397
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",0,10072.986413
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",0,10073.262395


In [64]:
amenity_within_isochrones = gpd.sjoin(schools_gdf,parks_isochrones,how = 'inner', predicate = 'within')
parks_isochrones['nearest_school_distance'] = calculate_nearest_amenity_distance(parks_detail, amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,job_count,nearest_metro_distance,nearest_school_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",0,10173.356557,24059.722548
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",0,10141.741316,24008.126927
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",0,10083.838397,23957.419887
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",0,10072.986413,23928.070679
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",0,10073.262395,23906.768645


In [65]:
amenity_within_isochrones = gpd.sjoin(landmarks_gdf,parks_isochrones,how = 'inner', predicate = 'within')
parks_isochrones['nearest_landmark_distance'] = calculate_nearest_amenity_distance(parks_detail, amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,job_count,nearest_metro_distance,nearest_school_distance,nearest_landmark_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",0,10173.356557,24059.722548,23875.412172
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",0,10141.741316,24008.126927,23852.362152
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",0,10083.838397,23957.419887,23792.903360
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",0,10072.986413,23928.070679,23789.298225
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",0,10073.262395,23906.768645,23797.330643


In [66]:
amenity_within_isochrones = gpd.sjoin(housing_gdf,parks_isochrones,how = 'inner', predicate = 'within')
parks_isochrones['nearest_residential_distance'] = calculate_nearest_amenity_distance(parks_detail, amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,job_count,nearest_metro_distance,nearest_school_distance,nearest_landmark_distance,nearest_residential_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",0,10173.356557,24059.722548,23875.412172,inf
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",0,10141.741316,24008.126927,23852.362152,inf
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",0,10083.838397,23957.419887,23792.903360,inf
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",0,10072.986413,23928.070679,23789.298225,inf
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",0,10073.262395,23906.768645,23797.330643,inf


In [67]:
amenity_within_isochrones = gpd.sjoin(MHA_gdf_clip,parks_isochrones,how = 'inner', predicate = 'within')
parks_isochrones['nearest_MHA_distance'] = calculate_nearest_amenity_distance(parks_detail, amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,GlobalID,geometry,centroid,isochrones,job_count,nearest_metro_distance,nearest_school_distance,nearest_landmark_distance,nearest_residential_distance,nearest_MHA_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",0,10173.356557,24059.722548,23875.412172,inf,inf
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",0,10141.741316,24008.126927,23852.362152,inf,inf
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",0,10083.838397,23957.419887,23792.903360,inf,inf
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",0,10072.986413,23928.070679,23789.298225,inf,inf
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",0,10073.262395,23906.768645,23797.330643,inf,inf


In [68]:
parks_isochrones.to_csv(f'park_deatails_{profile}_{travel_time}_Distance_Amentities.csv',index=True)

In [70]:
def calculate_nearest_park_distance(parks):
    # 列表存储每个公园到最近公园的距离
    distances = []
    
    # 对每个公园进行处理
    for _, park in parks.iterrows():
        park_point = park.geometry.centroid
        # 计算该公园到所有其他公园的距离
        distances_to_other_parks = parks.loc[parks.index != _].geometry.distance(park_point)
        # 找到最小距离
        nearest_distance = distances_to_other_parks.min() if not distances_to_other_parks.empty else float('inf')
        distances.append(nearest_distance)
    
    return distances

In [71]:
nearest_park_distances = calculate_nearest_park_distance(parks_detail)
parks_detail['nearest_park_distance'] = nearest_park_distances
parks_detail.head()
parks_detail.to_csv("parks_detail_park_accessibility_amentity_distance.csv", index=True)

In [72]:
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones,nearest_park_distance
50,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...",6.523181
1625,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.0938 6023678.225553),"POLYGON ((-13623401.623401 6038683.571267, -13...",9.128640
2591,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.819986 6023732.238333),"POLYGON ((-13616262.815776 6035189.127901, -13...",50.796062
992,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...",6.525327
1523,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.851761 6023768.188662),"POLYGON ((-13619588.485564 6033210.681069, -13...",22.683068


### Pandana

In [73]:
# 计算每个公园到最近的自行车网络的距离

In [74]:
# 1.加载自行车网络

import numpy as np
import osmnx as ox

# 使用osmnx从GraphML文件加载自行车网络数量
bike_network = ox.load_graphml('bike_network.graphml')
nodes,edges = ox.graph_to_gdfs(bike_network)

# 重置索引,将‘u’和‘v’转化为普通列
edges = edges.reset_index()

In [80]:
# 2.计算每个公园到所有自行车网络边的距离
def calculate_nearest_distance(point,edges):
    # 计算公园到所有自行车网络边的距离
    distance = [point.distance(edge) for edge in edges.geometry]
    return min(distance) if distance else float('inf')

In [81]:
# 3. 筛选时间范围内的自行车网络边
edges = edges.to_crs(epsg=projected_crs)
edges_within_isochrones = gpd.sjoin(edges, parks_isochrones, how='inner', predicate='within')
len(edges_within_isochrones)

2117

In [82]:
parks_detail['distance_to_nearest_bike_network'] = parks_detail.geometry.centroid.apply(lambda x: calculate_nearest_distance(x, edges_within_isochrones))
parks_detail.to_csv("parks_detail_accessibility_10_bike_network.csv", index = True)

In [ ]:
### 多推导几遍最短距离这件事,以及为什么要做最短距离